In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader,random_split,Dataset

In [2]:
l = []
with open("/content/drive/MyDrive/Data/Persian-WikiText-1.txt") as f :
  l = f.readlines()

In [3]:
import re
texts = [re.sub(r'[^\w]',' ',t) for t in l]
print(texts[40])
print("---")
texts = [re.sub(r'\s+',' ',t) for t in texts]
print(texts[40])

 
---
 


In [4]:
texts  = [t for t in texts if t not in [' ', '\n']]

In [5]:
texts = [re.sub(r'^\s','',t) for t in texts];
texts = [re.sub(r'\s$','',t) for t in texts];

In [6]:
texts[5]

'ویکی پدیا از پایان آوریل ۲۰۰۷ تا اکتبر ۲۰۱۹ یکی از ۱۰ وبگاه برتر جهان از لحاظ شمار بازدیدکنندگان بوده است که بیش از نیمی از بازدیدها به ویکی پدیای انگلیسی مربوط می شود'

In [7]:
from transformers import AutoTokenizer, AutoModelForMaskedLM,DataCollatorForLanguageModeling

In [8]:
torch.cuda.empty_cache()

In [9]:
model_name = 'HooshvareLab/bert-fa-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
max(len(t.split()) for t in texts)

1842

In [12]:
class Text(Dataset):
  def __init__(self, texts, tokenizer):
    super().__init__() ;
    self.texts = texts;
    self.tokenizer = tokenizer;
  def __len__(self): return len(self.texts);
  def __getitem__(self, index):
    text = self.texts[index];
    tokens = self.tokenizer(text,truncation=True,max_length=256,padding='max_length',return_tensors='pt');
    item = {key:val.squeeze(0) for key,val in tokens.items()};
    return item

In [13]:
dataset = Text(texts,tokenizer);
dataset[10]

{'input_ids': tensor([    2,  1379,  2880,  4207,  3212, 51278, 32824,  3500,  2789,  2867,
          2830, 51278, 32824,  4484, 13494,  6552,  1379, 18879,  2003,  2834,
          2800,  6012,  8278,  3158,  5613,  1379,  2791,  3626,  2829,  3877,
          4259,  4196,  2793,  3589, 51278, 32824,  2786,  3378,  1463,  6023,
          6912,  3032,  5667, 13994,  3671,  7959,  3052,  6166,  2017,  2800,
          2829,  3052, 30122,  4670,  2834,  6130,  4604,  2817,  3052,  6166,
          2017,  2800,  2786,  2844, 10155,  6130,  4604,  2871,  2834,  2829,
          4795,  6825, 13070, 35008,  1379,  3450,  7443,  5172,  2831,  5590,
          2834,  1379,  3497,  3923, 13494,  3912, 51278, 32824,  1379,  7233,
          2808, 12570, 47975,  3158,  2808,  2848,  2844,  8043,  3823,  2793,
          2817,  9908,  6715,  1379,  9258,  2988,  2871,  2831,  4484, 12570,
         47975,  3158,  6166,  2017,  4624,  2817,  1379,  3398,  2791,  8276,
          3052,  6166,  2017,  1379,  2

In [14]:
len(dataset)

212623

In [15]:
collate = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=True,mlm_probability=0.15);

In [16]:
train_dataset,val_dataset = random_split(dataset,[200000,12623])
train_loader = DataLoader(train_dataset,batch_size=4,shuffle=True,collate_fn=collate);
val_loader = DataLoader(val_dataset,batch_size=4,shuffle=False,collate_fn =collate);

In [17]:
model = AutoModelForMaskedLM.from_pretrained(model_name);
device = 'cuda'
model.to(device)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [53]:
optimizer = torch.optim.Adam(lr = 1e-5,params = model.parameters())
loss_function = nn.CrossEntropyLoss(ignore_index=-100);

In [54]:
b = 0;
model.train()
for x in train_loader :
  optimizer.zero_grad();
  input_ids = x['input_ids'].to(device);
  attention_mask = x['attention_mask'].to(device);
  labels = x['labels'].to(device);
  out = model(input_ids = input_ids,attention_mask = attention_mask,labels=labels);
  loss = out.loss
  loss.backward();
  optimizer.step();
  b+=1;
  if (b%10==0): print(f"Iteration {b} ---> Loss {round(loss.item(),4)}");


Iteration 10 ---> Loss 8.2909
Iteration 20 ---> Loss 8.0047
Iteration 30 ---> Loss 6.1657
Iteration 40 ---> Loss 3.8094
Iteration 50 ---> Loss 4.7057
Iteration 60 ---> Loss 3.9707
Iteration 70 ---> Loss 1.8695
Iteration 80 ---> Loss 2.0591
Iteration 90 ---> Loss 3.7987
Iteration 100 ---> Loss 2.7061
Iteration 110 ---> Loss 3.7735
Iteration 120 ---> Loss 3.2304
Iteration 130 ---> Loss 4.5588
Iteration 140 ---> Loss 2.7743
Iteration 150 ---> Loss 2.6107
Iteration 160 ---> Loss 2.1651
Iteration 170 ---> Loss 3.1968
Iteration 180 ---> Loss 1.8216
Iteration 190 ---> Loss 3.3911
Iteration 200 ---> Loss 2.0365
Iteration 210 ---> Loss 3.7345
Iteration 220 ---> Loss 2.6522
Iteration 230 ---> Loss 1.9828
Iteration 240 ---> Loss 3.5046
Iteration 250 ---> Loss 3.4611
Iteration 260 ---> Loss 2.6047
Iteration 270 ---> Loss 2.6107
Iteration 280 ---> Loss 2.9071
Iteration 290 ---> Loss 3.3612
Iteration 300 ---> Loss 2.3992
Iteration 310 ---> Loss 4.1753
Iteration 320 ---> Loss 1.9403
Iteration 330 ---

KeyboardInterrupt: 

In [32]:
tokenizer.convert_tokens_to_ids("[MASK]")

[3]

In [55]:
sentence = 'تهران پاییتخت [MASK] است'
model.eval();
with torch.no_grad():
  tokens= tokenizer(sentence,max_length=256,padding='max_length',truncation=True,return_tensors='pt');
  item = {key:val.squeeze(0) for key,val in tokens.items()};
  attention_mask = item['attention_mask'].unsqueeze(0).to(device);
  input_ids = item['input_ids'].unsqueeze(0).to(device)
  labels = torch.tensor([-100] * len(item['input_ids']),dtype=torch.int64).unsqueeze(0).to(device);
  out = model(attention_mask = attention_mask,input_ids=input_ids,labels=labels);
  logits = out.logits

In [57]:
input_ids

tensor([[   2, 3326, 7859, 5516,    3, 2806,    4,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [60]:
predict = logits[0,4,:].argmax()

In [61]:
predict

tensor(2927, device='cuda:0')

In [63]:
tokenizer.convert_ids_to_tokens([predict])

['ایران']

In [77]:
sentence = "انگلیس در قاره ی [MASK] واقع شده است"
model.eval();
with torch.no_grad():
  tokens= tokenizer(sentence,max_length=256,padding='max_length',truncation=True,return_tensors='pt');
  item = {key:val.squeeze(0) for key,val in tokens.items()};
  attention_mask = item['attention_mask'].unsqueeze(0).to(device);
  input_ids = item['input_ids'].unsqueeze(0).to(device)
  labels = torch.tensor([-100] * len(item['input_ids']),dtype=torch.int64).unsqueeze(0).to(device);
  out = model(attention_mask = attention_mask,input_ids=input_ids,labels=labels);
  logits = out.logits
input_ids = input_ids.squeeze(0);
masks = torch.tensor([i for i in range(len(input_ids)) if input_ids[i].item()==3])
predict = logits[0,masks,:].argmax()
tokenizer.convert_ids_to_tokens([predict])

['اسیا']

In [75]:
input_ids.squeeze(0)

tensor([   2, 5853, 2786, 8080, 1442,    3, 3473, 2871, 2806,    4,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [76]:
for i in range(len(input_ids.squeeze(0))):
  print(input_ids.squeeze(0)[i])

tensor(2, device='cuda:0')
tensor(5853, device='cuda:0')
tensor(2786, device='cuda:0')
tensor(8080, device='cuda:0')
tensor(1442, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3473, device='cuda:0')
tensor(2871, device='cuda:0')
tensor(2806, device='cuda:0')
tensor(4, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(